## Imports

In [399]:
import numpy as np
import math
import random

## Functions

In [408]:
def read_sudoku(path):
    """ Reading sudoku from file, there are 6 files in directory "sudoku" with 2 different puzzles
        each of them has 30, 45 or 55 empty cells.
    """
    with open(path) as file:
        sudoku = np.zeros((9, 9), dtype=int)
        for i in range(9):
            row = file.readline()
            for j in range(9):
                if row[j] == "x":
                    sudoku[i, j] = 0
                else:
                    sudoku[i, j] = int(row[j][0])

        return sudoku


def get_blocked(sudoku):
    """ Some of sudoku's cells are filled at the beginning. This function extracts them 
        to prevent making any changes on them
    """
    blocked = np.zeros((9, 9), dtype=bool)
    for i in range(9):
        for j in range(9):
            if sudoku[i, j] != 0:
                blocked[i, j] = True

    return blocked


def fill_random(sudoku):
    """ Filling empty cells with random numbers """
    for i in range(9):
        for j in range(9):
            if sudoku[i, j] == 0:
                sudoku[i, j] = random.randint(1, 9)

    return sudoku


def get_cost(sudoku):
    """ Getting cost of current state of algorithm. 1 unit one duplication of one number 
        in row, column or block
    """
    cost = 0

    # number of digit in every: row, column, block
    digits_rows = [[0 for _ in range(10)] for _ in range(9)]
    digits_columns = [[0 for _ in range(10)] for _ in range(9)]
    digits_blocks = [[0 for _ in range(10)] for _ in range(9)]

    for i in range(9):
        for j in range(9):
            digits_rows[i][sudoku[i, j]] += 1
            digits_columns[i][sudoku[j, i]] += 1
            digits_blocks[i][sudoku[3 * (i // 3) + (j // 3), 3 * (i % 3) + (j % 3)]] += 1

        for j in range(1, 10):
            if digits_rows[i][j] > 1:
                cost += digits_rows[i][j] - 1
            if digits_columns[i][j] > 1:
                cost += digits_columns[i][j] - 1
            if digits_blocks[i][j] > 1:
                cost += digits_blocks[i][j] - 1

    # digits_rows, digits_columns and digits_blocks are necessary for anneal function
    return cost, digits_rows, digits_columns, digits_blocks


def anneal(sudoku, t_max, multiplier, iterations):
    """ Function which makes simulated annealing. """
    
    # getting matrix of blocked numbers in sudoku
    blocked = get_blocked(sudoku)
    
    # filling empty cells with random numbers
    sudoku = fill_random(sudoku)

    # getting current costs and number of digits in rows, columns and blocks
    cost, digits_rows, digits_columns, digits_blocks = get_cost(sudoku)

    it = 1
    t = t_max
    while it < iterations and cost > 0:
        # getting random number which is duplicated in row, column or block and changing it 
        # to a random one which is not present in the row, column or block
        while True:
            # row, column or block
            to_update = random.choice(["row", "column", "block"])

            # which row, column or block
            num = random.randint(0, 8)

            # here will be saved all numbers duplicated and numbers which are not present
            too_many = []
            zeros = []
            if to_update == "row":
                for i in range(1, 10):
                    if digits_rows[num][i] == 0:
                        zeros.append(i)
                    elif digits_rows[num][i] > 1:
                        too_many.append(i)
            elif to_update == "column":
                for i in range(1, 10):
                    if digits_columns[num][i] == 0:
                        zeros.append(i)
                    elif digits_columns[num][i] > 1:
                        too_many.append(i)
            else:
                for i in range(1, 10):
                    if digits_blocks[num][i] == 0:
                        zeros.append(i)
                    elif digits_blocks[num][i] > 1:
                        too_many.append(i)

            # if there were any duplication in the row, column or block which have been chosen before
            if len(too_many) > 0:
                # getting random number to change
                to_change = random.choice(too_many)
                
                # getting random number which will be in place of the one chosen before
                to_increase = random.choice(zeros)

                # places in row, column or block which contain number equal to "to_change"
                places = []
                if to_update == "row":
                    for i in range(9):
                        if sudoku[num][i] == to_change:
                            places.append((num, i))
                elif to_update == "column":
                    for i in range(9):
                        if sudoku[i][num] == to_change:
                            places.append((i, num))
                else:
                    for i in range(9):
                        row = 3 * (num // 3) + (i // 3)
                        column = 3 * (num % 3) + (i % 3)
                        if sudoku[row][column] == to_change:
                            places.append((row, column))

                # getting place in matrix with chosen number (one of them can be blocked)
                cell = (0, 0)
                while places:
                    cell = random.choice(places)

                    if blocked[cell[0], cell[1]]:
                        places.remove(cell)
                    else:
                        break

                # making new sudoku matrix with changed cell
                new_sudoku = sudoku.copy()
                new_sudoku[cell[0], cell[1]] = to_increase

                # getting cost of new sudoku matrix
                new_cost, new_digits_rows, new_digits_columns, new_digits_blocks = get_cost(new_sudoku)

                # checking if sudoku should be updated
                if new_cost < cost or math.e ** ((cost - new_cost) / t) > random.random():
                    sudoku = new_sudoku
                    cost = new_cost
                    digits_rows = new_digits_rows
                    digits_columns = new_digits_columns
                    digits_blocks = new_digits_blocks
                break

        it += 1
        t *= multiplier

    # information about solving sudoku (cost == 0 means that sudoku is solved)
    print("After", it, "iterations cost is equal to", cost)
    
    return sudoku, it, cost


def solve_n_times(sudoku, n, t_max, multiplier, iterations):
    """ Many attempts show that simulated annealing not always return solved sudoku.
        To make some statistics this function is solving the same puzzle n times.
    """
    
    # statistics
    solved = 0
    mean_cost = 0
    mean_iterations = 0
    
    # solving n times
    for i in range(n):
        s = sudoku.copy()
        _, it, cost = anneal(sudoku, t_max, multiplier, iterations)
        
        if cost == 0:
            solved += 1
            mean_iterations += it
        else:
            mean_cost += cost
            
    if solved < n:
        mean_cost /= (n - solved)
    if solved != 0:
        mean_iterations /= solved
        
    # printing results
    print()
    print(n, "attempts. Solved", solved, "times.")
    print("Average number of iterations needed to solve is", int(mean_iterations))
    print("Average cost when algorithm failed to solve is", "{0:.2f}".format(mean_cost))

## Tests

### First example

In [411]:
# 30 empty cells in sudoku
t_max = 5
multiplier = 0.996
iterations = 1000

sudoku = read_sudoku("sudoku/ex1.txt")
solve_n_times(sudoku, 20, t_max, multiplier, iterations)

After 67 iterations cost is equal to 0
After 598 iterations cost is equal to 0
After 252 iterations cost is equal to 0
After 590 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 210 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 500 iterations cost is equal to 0
After 288 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 404 iterations cost is equal to 0
After 199 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 630 iterations cost is equal to 0
After 163 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 415 iterations cost is equal to 0
After 287 iterations cost is equal to 0
After 284 iterations cost is equal to 0

20 attempts. Solved 14 times.
Average number of iterations needed to solve is 349
Average cost when algorithm failed to solve is 2.00


In [413]:
# 45 empty cells in sudoku
t_max = 5
multiplier = 0.998
iterations = 1000

sudoku = read_sudoku("sudoku/ex2.txt")
solve_n_times(sudoku, 20, t_max, multiplier, iterations)

After 153 iterations cost is equal to 0
After 1000 iterations cost is equal to 5
After 518 iterations cost is equal to 0
After 483 iterations cost is equal to 0
After 365 iterations cost is equal to 0
After 283 iterations cost is equal to 0
After 500 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 3
After 323 iterations cost is equal to 0
After 443 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 488 iterations cost is equal to 0
After 390 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 7
After 536 iterations cost is equal to 0
After 633 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2

20 attempts. Solved 12 times.
Average number of iterations needed to solve is 426
Average cost when algorithm failed to solve is 3.12


In [414]:
# 55 empty cells in sudoku
t_max = 5
multiplier = 0.999
iterations = 1500

sudoku = read_sudoku("sudoku/ex3.txt")
solve_n_times(sudoku, 20, t_max, multiplier, iterations)

After 1500 iterations cost is equal to 4
After 640 iterations cost is equal to 0
After 1500 iterations cost is equal to 5
After 1500 iterations cost is equal to 3
After 282 iterations cost is equal to 0
After 275 iterations cost is equal to 0
After 289 iterations cost is equal to 0
After 816 iterations cost is equal to 0
After 310 iterations cost is equal to 0
After 1500 iterations cost is equal to 2
After 592 iterations cost is equal to 0
After 1500 iterations cost is equal to 3
After 920 iterations cost is equal to 0
After 1500 iterations cost is equal to 8
After 493 iterations cost is equal to 0
After 1500 iterations cost is equal to 5
After 293 iterations cost is equal to 0
After 1500 iterations cost is equal to 2
After 1500 iterations cost is equal to 5
After 1500 iterations cost is equal to 2

20 attempts. Solved 10 times.
Average number of iterations needed to solve is 491
Average cost when algorithm failed to solve is 3.90


### Second Example

In [416]:
# 30 empty cells in sudoku
t_max = 5
multiplier = 0.9965
iterations = 1000

sudoku = read_sudoku("sudoku/ex4.txt")
solve_n_times(sudoku, 20, t_max, multiplier, iterations)

After 98 iterations cost is equal to 0
After 234 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 4
After 1000 iterations cost is equal to 2
After 408 iterations cost is equal to 0
After 405 iterations cost is equal to 0
After 447 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 490 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 476 iterations cost is equal to 0
After 1000 iterations cost is equal to 4
After 1000 iterations cost is equal to 4
After 619 iterations cost is equal to 0
After 1000 iterations cost is equal to 3
After 359 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 4

20 attempts. Solved 9 times.
Average number of iterations needed to solve is 392
Average cost when algorithm failed to solve is 2.82


In [417]:
# 45 empty cells in sudoku
t_max = 5
multiplier = 0.9965
iterations = 1000

sudoku = read_sudoku("sudoku/ex5.txt")
solve_n_times(sudoku, 20, t_max, multiplier, iterations)

After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 4
After 357 iterations cost is equal to 0
After 1000 iterations cost is equal to 3
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 240 iterations cost is equal to 0
After 1000 iterations cost is equal to 4
After 432 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 470 iterations cost is equal to 0
After 414 iterations cost is equal to 0

20 attempts. Solved 5 times.
Average number of iterations needed to solve is 382
Average cost when algorithm failed to solve is 2.33


In [419]:
# 55 empty cells in sudoku
t_max = 5
multiplier = 0.997
iterations = 1000

sudoku = read_sudoku("sudoku/ex6.txt")
solve_n_times(sudoku, 20, t_max, multiplier, iterations)

After 1000 iterations cost is equal to 4
After 742 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 4
After 512 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 449 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 278 iterations cost is equal to 0
After 436 iterations cost is equal to 0
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 1000 iterations cost is equal to 2
After 673 iterations cost is equal to 0
After 256 iterations cost is equal to 0
After 473 iterations cost is equal to 0
After 441 iterations cost is equal to 0

20 attempts. Solved 9 times.
Average number of iterations needed to solve is 473
Average cost when algorithm failed to solve is 2.36
